In [1]:
from murpheus import DataLoading
from dask.distributed import Client
import dask.dataframe as dd
from emoji import UNICODE_EMOJI
import re
from sklearn.model_selection import train_test_split
import emoji

import tensorflow as tf
from tqdm.notebook import tqdm

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:41165 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.69 GB


In [ ]:
tweet_df = DataLoading().get_twitter_data_as_bags('../../data/06/01/**/*').to_dataframe()
tweet_df

In [ ]:
# selecting only the english language
tweet_df = tweet_df[tweet_df['lang'] == 'en']
tweet_df

In [ ]:
# removing all of the tweets that are truncated
tweet_df['is_full_tweet'] = tweet_df.apply(lambda x: x['text'][-1] != '…', axis=1, meta=bool)
tweet_df = tweet_df[tweet_df['is_full_tweet']]
tweet_df

In [ ]:
# checking to see if there's an emoji in the text
def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            return False
    return bool(count)

tweet_df = tweet_df[tweet_df['text'].apply(is_emoji, meta=bool)]
tweet_df

In [ ]:
# getting emoji's from the texts and how many tweets there are

def extract_emojis(s):
    return ''.join(c for c in s if c in UNICODE_EMOJI)

tweet_df['emojis'] = tweet_df.apply(lambda x: extract_emojis(x['text']), axis=1, meta=str)
tweet_df['emojis_count'] = tweet_df.apply(lambda x: len(x['emojis']), meta=int, axis=1)
tweet_df

In [ ]:
# removing @RT from all of the tweets
tweet_df['text_without_retweets'] = tweet_df['text'].apply(lambda x: re.sub(r'RT @(.+?):', '', x), meta=str)
tweet_df

In [ ]:
# getting only tweets with one emoji
tweet_df = tweet_df[tweet_df['emojis_count'] == 1]
tweet_df

In [ ]:
tweet_df[['text_without_retweets', 'emojis']].to_csv('text_emoji_data/text_emoji_data-*.csv')

In [3]:
# process day
# checking to see if there's an emoji in the text
def is_emoji(s):
    count = 0
    for emoji in UNICODE_EMOJI:
        count += s.count(emoji)
        if count > 1:
            return False
    return bool(count)

def extract_emojis(s):
    return ''.join(c for c in s if c in UNICODE_EMOJI)

for day in tqdm([1,2,4, 5, 6, 7, 8, 9, 10]):
    day = f'0{day}' if len(str(day)) == 1 else str(day)
    tweet_df = DataLoading().get_twitter_data_as_bags(f'../../data/06/{day}/**/*').to_dataframe()
    # selecting only the english language
    tweet_df = tweet_df[tweet_df['lang'] == 'en']
    # removing all of the tweets that are truncated
    tweet_df['is_full_tweet'] = tweet_df.apply(lambda x: x['text'][-1] != '…', axis=1, meta=bool)
    tweet_df = tweet_df[tweet_df['is_full_tweet']]
    tweet_df = tweet_df[tweet_df['text'].apply(is_emoji, meta=bool)]
    tweet_df['emojis'] = tweet_df.apply(lambda x: extract_emojis(x['text']), axis=1, meta=str)
    tweet_df['emojis_count'] = tweet_df.apply(lambda x: len(x['emojis']), meta=int, axis=1)
    tweet_df['text_without_retweets'] = tweet_df['text'].apply(lambda x: re.sub(r'RT @(.+?):', '', x), meta=str)
    tweet_df = tweet_df[tweet_df['emojis_count'] == 1]
    tweet_df[['text_without_retweets', 'emojis']].to_csv(f'text_emoji_data/text_emoji_data-{day}-*.csv')

  0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
dl = DataLoading()
del dl
dl = DataLoading()
dl

In [ ]:
tweet_df = dd.read_csv('text_emoji_data/text_emoji_data-*.csv')#[['text', 'emojis']]
tweet_df

In [ ]:
tweet_df.head()

In [ ]:
tweet_df['text'] = tweet_df['text'].apply(lambda x: re.sub(r'RT @(.+?):', '', x), meta=str)
tweet_df

In [ ]:
tweet_df[['text', 'emojis']].to_csv('text_emoji_data-cleaned/text_emoji_data-*.csv', index=False)

In [ ]:
tweet_df = dd.read_csv('text_emoji_data-cleaned/text_emoji_data-*.csv')#.persist()
tweet_df

In [ ]:
tweet_df.head()

In [ ]:
emoji_counts = tweet_df.groupby('emojis').count().compute()
emoji_counts

In [ ]:
emoji_counts = emoji_counts.sort_values('text', ascending=False).head(10)
emoji_counts = emoji_counts.reset_index()
emoji_counts

In [ ]:
tweet_df_training_data = tweet_df.loc[tweet_df.emojis.isin(emoji_counts.emojis)]
tweet_df_training_data.shape[0].compute()

In [ ]:
_demojifier_regex = r':.+?:'
_username_regex = r'@.+? '

def _remove_emojis(string: str):
    string = emoji.demojize(string)
    return re.sub(_demojifier_regex, '', string)

def filter_emoji(twitter_dataframe):
    twitter_dataframe['text'] = twitter_dataframe['text'].apply(_remove_emojis, meta=str)
    return twitter_dataframe

def _remove_username(string: str):
    return re.sub(_username_regex, '', string)

def filter_username(twitter_dataframe):
    twitter_dataframe['text'] = twitter_dataframe['text'].apply(_remove_username, meta=str)
    return twitter_dataframe

In [ ]:
tweet_df_training_data = filter_emoji(tweet_df_training_data)
tweet_df_training_data = filter_username(tweet_df_training_data)
tweet_df_training_data

In [ ]:
tweet_df_training_data = tweet_df_training_data.compute()
tweet_df_training_data

In [ ]:
labels = list(set(tweet_df_training_data['emojis']))
encode_key = {labels[i]:i for i in range(len(labels))}
encode_key

In [ ]:
tweet_df_training_data['encoded_emojis'] = tweet_df_training_data.apply(lambda x: encode_key[x['emojis']], axis=1)#.head()
tweet_df_training_data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tweet_df_training_data['text'], tweet_df_training_data['encoded_emojis'])
# y_train = tf.one_hot(y_train, depth=len(labels))
# y_test = tf.one_hot(y_test, depth=len(labels))

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)

In [ ]:
y_train

In [ ]:
import autokeras as ak

# Initialize the text classifier.
clf = ak.TextClassifier(
    overwrite=True,
    max_trials=5)  # It only tries 1 model as a quick demo.
# Feed the text classifier with training data.
clf.fit(train_dataset, epochs=2)

In [ ]:
# Predict with the best model.
# predicted_y = clf.predict(x_test)
# Evaluate the best model with testing data.
print(clf.evaluate(test_dataset))